<a href="https://colab.research.google.com/github/Walaga/Pothole-detector/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title [IMPORTANT] Run Cell to Set up Environment {display-mode: "form"}
%%capture
# 1. Create a directory named `object_detection`, this is where the model files will be stored.
# 2. Download and install the Tensorflow Object Detection API models using [Git](https://en.wikipedia.org/wiki/Git) into the `object_detection` folder.
# 3. Install the required libraries for the API.
# 4. Install TF-Slim, a lightweight library for defining, training and evaluating complex models in TensorFlow.

# We shall import the os library, it provides a number of functions that allow 
# interaction with the operating environment.
import os
import json
import sys
import glob
import urllib
import io
import xml.etree.ElementTree as ET
import argparse
from pathlib import Path
import numpy as np
from shutil import copyfile
import csv
import pandas as pd
from PIL import Image
from tqdm import tqdm
import tensorflow.compat.v1 as tf
from collections import namedtuple

# 1. Create a directory named detection
!mkdir detection
os.chdir('detection')

# 2a. Download and install the Tensorflow Object Detection API models using Git into the object_detection folder,
# we use git command to get the source files from GitHub, git is preinstalled in the Google Colab environment.
!git clone --depth 1 https://github.com/tensorflow/models.git

# 2b. Add the folder to the path, this allows to import 
#     scripts as we do with install libraries
os.environ['PYTHONPATH'] += ':'+'/content/detection/models'

# 3. Install the required libraries for the API. The libraries are installed
#    from the setup.py.

# 3a. First we shall move into research/models, this is where the setup.py script
#     is stored.
%cd models/research

# Import dataset preparations from the object detection folder
from object_detection.utils import dataset_util

# 3b. This commands the environment to run the setup.py script if it exists.
!pip install .

# 4. Install TF-Slim, a lightweight library for defining, training and evaluating complex models in TensorFlow.
!pip install tf_slim

# 5a. Install the protobuf dependencies.
!protoc object_detection/protos/*.proto --python_out=.;
pwd = os.getcwd();

# 5b. Add TF_slim to the system path.
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim';

# 6. Move back to the object_detection folder.
%cd ../../

def split_indices(x, train=0.8, test=0.0, validate=0.2, shuffle=True):
    """
      Returns the indices at which the data is split.
    """
    # split training data
    n = len(x)
    v = np.arange(n)
    if shuffle:
        np.random.shuffle(v)

    i = round(n * train)  # train
    j = round(n * test) + i  # test
    k = round(n * validate) + j  # validate
    return v[:i], v[i:j], v[j:k]  # return indices

def split_files(file_names,train=0.8, test=0.2, validate=0.0):
    """
      Split the files provided according to the specified distributions.

      file_names  this is a list of file names that are split.
      train       the distribution for the train files, 0 <= x <= 1
      test        the distribution for the test files, it should complement train to add to one.
                  i.e. if train is 0.8, test should be 0.2 such that 0.8 + 0.2 = 1.0
      validate    this is the distribution for the validation set, it should also 
                  complement the train and test values i.e if train is 0.8, test is 0.1, validate
                  should be 0.1 such that 0.8 + 0.1 + 0.1 = 1.0

      Returns:
        (tuple) train, test, val
                train   a list containing the file names of the train set
                test    a list containing the file names of the test set
                val     a list containing the file name of the validation set
    """
    # split training data
    file_name = list(filter(lambda x: len(x) > 0, file_names))
    file_name = sorted(file_name)
    i, j, k = split_indices(file_names, train=train, test=test, validate=validate)
    train = []
    test = []
    val = []
    datasets = {'train': i, 'test': j, 'val': k}
    for key, item in datasets.items():
        if item.any():
            for ix in item:
                if key == 'train':
                    train.append(file_names[ix])
                if key == 'test':
                    test.append(file_names[ix])
                if key == 'val':
                    val.append(file_names[ix])

    return train, test, val

def json_to_csv(file_names, images_dir, labels_path, annotations_dir, label_file_name):
  """
  Converts a JSON file to a csv file.

  file_names       list of file names.
  images_dir       path to the images directory
  labels_path      is the path to the labels JSON file.
  annotations_dir  is the directory in which the annotations will be stored.
  label_file_name  is the name of the output .csv file i.e. labels_train.csv
  """
  # Load coco file
  f = open(labels_path, 'r')
  COCO_DATA = json.load(f)
  f.close()

  images = COCO_DATA["images"]
  annotations = COCO_DATA["annotations"]
      
  # Generating the csv in the annotations folder under data directory. (Ideally)
  csv_file_name = os.path.join(str(annotations_dir), label_file_name)

  class_name = 'pothole' # Normally, there will be more than one class, extract accordingly.

  with open(csv_file_name, 'w') as csv_label_file:
    f = csv.writer(csv_label_file)
    f.writerow(['file_name', 'width', 'height', 'class', 'xmin', 'ymin', 
                'xmax', 'ymax'])

    for file_name in tqdm(file_names, desc = "Processing CSV"):
        id = None
        for image in images:
          if file_name == image['file_name']:
            id = image['id']

        im = Image.open(os.path.join(images_dir, file_name))
        width, height = im.size

        for annotation in annotations:
            if id == annotation['image_id']:
                bbox = annotation['bbox']

                # COCO bbox label format: [xmin, ymin, width, height]
                xmin = bbox[0]
                xmax = bbox[0] + bbox[2]
                ymin = bbox[1]
                ymax = bbox[1] + bbox[3]

                # Write to .csv file.
                f.writerow([file_name, width, height, class_name, xmin, ymin, xmax, ymax])  

def create_pbtxt(annotations_dir):
  """
    Creates a pbtxt file.

    TensorFlow requires a label map, which maps each of the used labels 
    to an integer values. This label map is used both by the training and detection 
    processes. Notice the labels are one-indexed i.e. start at 1 (one).

    Example:
    # example.pbtxt
    item {
      id: 1
      name: 'cat'
    }

    item {
      id: 2
      name: 'dog'
    }

    The file is stored under the annotations folder.
  """
  # Create the label map
  label_map_path = os.path.join(annotations_dir, "label_map.pbtxt")
  pbtxt_content = ""

  class_name = 'pothole' # Could be more than one class name.

  pbtxt_content = (
      pbtxt_content
      + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(1, class_name)
  )
  pbtxt_content = pbtxt_content.strip()
  with open(label_map_path, "w") as f:
      f.write(pbtxt_content)

def move_files(files, source, dest):
  """Move files from the source directory to the destination directory."""
  for filename in files:
    copyfile(os.path.join(source, filename),
                 os.path.join(dest, filename))
    
def class_text_to_int(row_label):
  if row_label == 'pothole': # the respective class_name
    return 1
  else:
    None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def create_tfrecord(labels_file, images_dir, annotations_dir):
  """
    Create a tfrecord from the .csv file.
  """
  csv_name = labels_file.split('/')[-1][:-4]
  writer = tf.io.TFRecordWriter(os.path.join(annotations_dir, csv_name + '.record'))
  path = os.path.join(images_dir)
  examples = pd.read_csv(labels_file)
  grouped = split(examples, 'file_name')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
  writer.close()
  print('Successfully created the TFRecords: {}'.format(csv_name + '.record'))

# The training helper function.
def train_model(path):
  """
    This is a helper function to aid with the training.
  """
  !python workspace/data/model_main_tf2.py \
    --model_dir=workspace/data/training \
    --pipeline_config_path={path}

# The model exporter helper function.
def export_model(config_path, checkpoint_dir, export_dir):
  """
    This function calls the model exporter script that creates the graph format
    of the model that can be exported.
  """
  !python workspace/data/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path {config_path} \
    --trained_checkpoint_dir {checkpoint_dir} \
    --output_directory {export_dir}


# Finally, we prepare the workspace folders as discussed in the last notebook. 
# Refer to it for details.
# Let's go ahead and create the folders.
!mkdir workspace
!mkdir workspace/data
!mkdir workspace/data/annotations
!mkdir workspace/data/images
!mkdir workspace/data/images/train
!mkdir workspace/data/images/test 
!mkdir workspace/data/pre-trained-model
!mkdir workspace/data/training

# Copy the model_main.py into the object_detection/workspace/data
# This is to aid with the training later on.
copyfile('/content/detection/models/research/object_detection/model_main_tf2.py',\
         '/content/detection/workspace/data/model_main_tf2.py')

copyfile('/content/detection/models/research/object_detection/exporter_main_v2.py', \
         '/content/detection/workspace/data/exporter_main_v2.py')

In [3]:
%cd ..

/content


In [5]:
! git clone https://github.com/Walaga/detector.git

fatal: destination path 'detector' already exists and is not an empty directory.


In [10]:
%cd ./annotations/

/content/detection/workspace/data/annotations


In [11]:
!unzip /content/detector/files.zip

Archive:  /content/detector/files.zip
  inflating: test.csv                
  inflating: train.csv               


In [12]:
annotations_dir = r'/content/detection/workspace/data/annotations'
create_pbtxt(annotations_dir)

In [13]:
%cd ..

/content/detection/workspace/data


In [14]:
# 1. Download the pretrained weights of your choice from TensorFlow’s detection model zoo.
PRE_TRAINED_MODEL_DIR = r'/content/detection/workspace/data/pre-trained-model'
MODEL = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'

!wget -P {PRE_TRAINED_MODEL_DIR} http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -zxvf {PRE_TRAINED_MODEL_DIR}/{MODEL} -C {PRE_TRAINED_MODEL_DIR} # Extract to a given directory

--2020-10-22 18:40:07--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.12.240, 2607:f8b0:4004:807::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.12.240|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘/content/detection/workspace/data/pre-trained-model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M   237MB/s    in 1.0s    

2020-10-22 18:40:08 (237 MB/s) - ‘/content/detection/workspace/data/pre-trained-model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco1

In [15]:
%cd ./annotations/

/content/detection/workspace/data/annotations


In [16]:
!unzip /content/detector/tfrecords.zip

Archive:  /content/detector/tfrecords.zip
  inflating: train.record            
  inflating: test.record             


In [17]:
%cd ..

/content/detection/workspace/data


In [18]:
%cd ./images

/content/detection/workspace/data/images


In [19]:
%cd ./test/

/content/detection/workspace/data/images/test


In [20]:
!unzip /content/detector/test.zip

Archive:  /content/detector/test.zip
  inflating: pothole 2 (1).jpg       
  inflating: pothole 2 (2).jpg       
  inflating: pothole 2 (3).jpg       
  inflating: pothole 2 (4).jpg       
  inflating: pothole 2 (5).jpg       
  inflating: pothole 2 (6).jpg       
  inflating: pothole 2 (7).jpg       
  inflating: pothole 2 (8).jpg       
  inflating: pothole 2 (9).jpg       
  inflating: pothole 2 (10).jpg      
  inflating: pothole 2 (11).jpg      
  inflating: pothole 2 (12).jpg      
  inflating: pothole 2 (13).jpg      
  inflating: pothole 2 (14).jpg      
  inflating: pothole 2 (15).jpg      
  inflating: pothole 2 (16).jpg      
  inflating: pothole 2 (17).jpg      
  inflating: pothole 2 (18).jpg      
  inflating: pothole 2 (19).jpg      
  inflating: pothole 2 (20).jpg      
  inflating: pothole 2 (21).jpg      
  inflating: pothole 2 (22).jpg      
  inflating: pothole 2 (23).jpg      
  inflating: pothole 2 (24).jpg      
  inflating: pothole 2 (25).jpg      
  inflating: 

In [21]:
%cd ..

/content/detection/workspace/data/images


In [22]:
%cd ./train

/content/detection/workspace/data/images/train


In [23]:
!unzip /content/detector/train.zip

Archive:  /content/detector/train.zip
  inflating: pothole (1).jpg         
  inflating: pothole (2).jpg         
  inflating: pothole (3).jpg         
  inflating: pothole (5).jpg         
  inflating: pothole (6).jpg         
  inflating: pothole (7).jpg         
  inflating: pothole (8).jpg         
  inflating: pothole (9).jpg         
  inflating: pothole (10).jpg        
  inflating: pothole (11).jpg        
  inflating: pothole (12).jpg        
  inflating: pothole (13).jpg        
  inflating: pothole (14).jpg        
  inflating: pothole (15).jpg        
  inflating: pothole (16).jpg        
  inflating: pothole (17).jpg        
  inflating: pothole (18).jpg        
  inflating: pothole (19).jpg        
  inflating: pothole (20).jpg        
  inflating: pothole (21).jpg        
  inflating: pothole (22).jpg        
  inflating: pothole (23).jpg        
  inflating: pothole (25).jpg        
  inflating: pothole (26).jpg        
  inflating: pothole (27).jpg        
  inflating:

In [ ]:
# 1. Run the command below to run tensorboard, --logdir should point to the folder that stores the 
#    checkpoints during training, in this case it is the training folder under the data directory.
%load_ext tensorboard
%tensorboard --logdir workspace/data/training

In [26]:
%cd ..


/content/detection/workspace/data/images


In [24]:
config_path ='/content/detection/workspace/data/pre-trained-model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config'
train_model(config_path)

python3: can't open file 'workspace/data/model_main_tf2.py': [Errno 2] No such file or directory


In [27]:
%cd ..

/content/detection/workspace/data


In [ ]:
# You have to give the script two required arguments
# something like below;
#!python model_main_tf2.py --model_dir=workspace/data/training --pipeline_config_path={path}
!python model_main_tf2.py 

In [ ]:
!python model_main_tf2.py --model_dir=/content/detection/workspace/data/training --pipeline_config_path={'/content/detection/workspace/data/pre-trained-model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config'}

2020-10-22 19:14:42.841294: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-22 19:14:44.738890: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-22 19:14:44.775053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-22 19:14:44.775622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-22 19:14:44.775661: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-22 19:14:44.777295: I tensorflow/stream_executor/platform/default

In [ ]:
path = r'/content/detection/workspace/data/pre-trained-model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config'
checkpoint_dir = r'/content/detection/workspace/data/training'
export_dir = r